Generating final model to be used in the application with the optimized parameters from the optimization.

In [1]:
import json

data_file = open("./training_data_set.json")
raw_data = json.load(data_file)
data_file.close()

sites = raw_data.get("sitesAndTags", [])
print("Quantidade total de sites:", len(sites))

Quantidade total de sites: 200


In [2]:
from sklearn.model_selection import train_test_split
import requests
from bs4 import BeautifulSoup

# Function to remove tags
def remove_tags(html):
    soup = BeautifulSoup(html, "html.parser")
  
    for data in soup(['style', 'script']):
        data.decompose()
  
    return ' '.join(soup.stripped_strings)

sites_contents = []
y_sites = []

for [url, usefulness] in sites:
    r = requests.get(url)
    filtered_content = remove_tags(r.content)
    sites_contents.append(filtered_content)
    y_sites.append(usefulness)
    print(url)

https://www.bigboygames.com.br/jogo-fallen-legion-revenants-vanguard-edition-nintendo-switch
https://www.bigboygames.com.br/world-of-warriors-ps4-4955-p993556
https://www.bigboygames.com.br/jogo-monster-hunter-world-seminovo-ps4-11807-br-p1006227
https://www.bigboygames.com.br/jogo-back-4-blood-xbox-one
https://www.bigboygames.com.br/jogo-far-cry-6-xbox-one-p1006787
https://www.bigboygames.com.br/controle-baseus-sem-fio-transparente-seminovo-nintendo-switch
https://www.bigboygames.com.br/cartao-xbox-live-brasil-r200-5598-p995690
https://www.bigboygames.com.br/case-zelda-botw-seminovo-nintendo-switch-lite-15816
https://www.bigboygames.com.br/console-nintendo-switch-lite-amarelo-seminovo-16101
https://www.bigboygames.com.br/case-protetora-nintendo-swtich-lite-cinza-amarelo-14940
https://www.ibyte.com.br/jogo-hades-xbox/p
https://www.ibyte.com.br/marvel-s-spider-man-miles-morales-ps5/p
https://www.ibyte.com.br/game-fifa-18-xbox-one/p
https://www.shockgames.com.br/produto/horizon-zero-dawn

In [3]:
%pip install nltk
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


[nltk_data] Downloading package stopwords to /home/alps2/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(sites_contents, y_sites, random_state = 1337)

Optimized parameters:
clf__bootstrap: False
clf__max_depth: 40
clf__min_samples_leaf: 1
clf__min_samples_split: 10
clf__n_estimators: 200
tfidf__use_idf: False
vect__max_features: 10000
vect__min_df: 0.005
vect__ngram_range: (1, 2)

Stats for optimized random forest
Precision: [0.95652174 0.81481481]
recall: [0.81481481 0.95652174]
f1: [0.88 0.88]

In [5]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

chosen_model_pipeline = Pipeline([
    ('vect', CountVectorizer(max_features=10000, min_df=0.005, ngram_range=(1, 2), stop_words=stopwords.words('portuguese'))),
    ('tfidf', TfidfTransformer(use_idf=False)),
    ('clf', RandomForestClassifier(bootstrap=False, max_depth=40, min_samples_leaf=1, min_samples_split=10, n_estimators=200)),
])

chosen_model_pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(max_features=10000, min_df=0.005,
                                 ngram_range=(1, 2),
                                 stop_words=['a', 'à', 'ao', 'aos', 'aquela',
                                             'aquelas', 'aquele', 'aqueles',
                                             'aquilo', 'as', 'às', 'até', 'com',
                                             'como', 'da', 'das', 'de', 'dela',
                                             'delas', 'dele', 'deles', 'depois',
                                             'do', 'dos', 'e', 'é', 'ela',
                                             'elas', 'ele', 'eles', ...])),
                ('tfidf', TfidfTransformer(use_idf=False)),
                ('clf',
                 RandomForestClassifier(bootstrap=False, max_depth=40,
                                        min_samples_split=10,
                                        n_estimators=200))])

Model persistence. The model can be loaded from the generated pickle file to be used in running python processes without the need of retraining it everytime.

In [6]:
from joblib import dump

dump(chosen_model_pipeline, './persistence/classifier_model.joblib')

# Can be loaded with:
# from joblib import load

# classifier_model = ('./persistence/classifier_model.joblib')
# predictions = classifier_model.predict(url_list)

['./persistence/classifier_model.joblib']